In [1]:
pip install langchain


  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     ---------------------------------------- 50.7/50.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/812.8 kB ? eta -:--:--
   --------------------------------------  809.0/812.8 kB 25.8 MB/s eta 0:00:01
   --------------------------------------- 812.8/812.8 kB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/366.0 kB ? eta -:--:--
   ---------------------------------------- 366.0/366.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 60.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/276.8 kB ? eta -:--:--
   --------------------------------------- 276.8/276.8 kB 17.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
   ---------------------------------------- 87.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.16.0 requires google-api-python-client>=1.6.7, which is not installed.
tf-models-official 2.16.0 requires immutabledict, which is not installed.
tf-models-official 2.16.0 requires kaggle>=1.3.9, which is not installed.
tf-models-official 2.16.0 requires oauth2client, which is not installed.
tf-models-official 2.16.0 requires sentencepiece, which is not installed.
tf-models-official 2.16.0 requires seqeval, which is not installed.
tf-models-official 2.16.0 requires tensorflow-hub>=0.6.0, which is not installed.
tf-models-official 2.16.0 requires tensorflow-model-optimization>=0.4.1, which is not installed.
tf-models-official 2.16.0 requires tensorflow-text~=2.16.1, which is not installed.
apache-beam 2.55.0 requires crcmod<2.0,>=1.7, which is not installed.
apache-beam 2.55.0 requires fastavro<

In [26]:
from bs4 import BeautifulSoup

In [15]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a web scraping specialist."),
    ("user", "{input}")
])

In [19]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [20]:
chain = prompt | llm | output_parser

# Retrieval Chain 

In [27]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.tutorialspoint.com/beautiful_soup/beautiful_soup_quick_guide.htm")

docs = loader.load()

In [28]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [29]:
pip install faiss-cpu


   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.5 MB 13.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/14.5 MB 32.1 MB/s eta 0:00:01
   -------------- ------------------------- 5.4/14.5 MB 43.2 MB/s eta 0:00:01
   --------------------- ------------------ 7.9/14.5 MB 45.7 MB/s eta 0:00:01
   --------------------------- ------------ 10.1/14.5 MB 46.0 MB/s eta 0:00:01
   -------------------------------- ------- 11.6/14.5 MB 50.4 MB/s eta 0:00:01
   --------------------------------- ------ 12.3/14.5 MB 40.9 MB/s eta 0:00:01
   -------------------------------------- - 13.9/14.5 MB 38.5 MB/s eta 0:00:01
   ---------------------------------------- 14.5/14.5 MB 38.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [32]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [33]:
response = retrieval_chain.invoke({"input": "What is Beautiful Soup used for?"})
print(response["answer"])

Beautiful Soup is a Python library used for parsing and scraping HTML and XML documents. It provides a simple and easy-to-use way to navigate and search through the contents of web pages, documents, and other structured content. Here are some common use cases for Beautiful Soup:

1. Web scraping: Beautiful Soup can be used to extract data from websites, such as prices, reviews, or news articles, by navigating through the HTML structure of the page.
2. Data mining: The library can be used to extract large amounts of data from websites, which can then be processed and analyzed using Python's data science libraries.
3. Web automation: Beautiful Soup can be used to automate tasks on a website, such as filling out forms or clicking buttons, by simulating user interaction with the HTML structure of the page.
4. Reverse engineering: The library can be used to analyze the structure and content of websites, including the use of hidden elements or JavaScript-generated content, which may not be v

In [34]:
response = retrieval_chain.invoke({"input": "What is the code to parse a webpage?"})
print(response["answer"])

To parse a webpage using Beautiful Soup, you can use the following code as a starting point:
```python
from bs4 import BeautifulSoup

# Load the webpage into a string
url = "https://www.example.com"
page = requests.get(url).text

# Create a Beautiful Soup object
soup = BeautifulSoup(page, "html.parser")

# Find the element you want to parse
element = soup.find("div", class_="my-class")

# Do something with the element
print(element.name)
```
In this code:

* `requests` is a Python package for making HTTP requests. You can replace `requests.get()` with any other way you have of loading the webpage, such as using `requests.get()` with a URL parameter.
* The string `page` contains the HTML content of the webpage.
* The `BeautifulSoup()` constructor takes the HTML content as input and returns a `BeautifulSoup` object.
* The `find()` method searches for an element on the page that matches the given selector (in this case, a `<div>` element with a class of `"my-class"`). If the method finds 